#### EX4: 

- Read some transformer papers and implement one additional feature or change that people seem to use. 
- Does it improve the performance of your GPT?

Let's go for: Flash Attention https://arxiv.org/pdf/2205.14135.pdf
Based on nanoGPT https://github.com/karpathy/nanoGPT/blob/master/model.py

#### Results:

- Even though the gpt model that used flash attention trained slightly faster
- it's val loss decreased from 1.48 to 1.58 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
#!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu117 -U

In [4]:
assert hasattr(torch.nn.functional, 'scaled_dot_product_attention')

In [5]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5001
eval_interval = 500
learning_rate = 3e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
eval_iters = 200
n_embd = 384
n_head = 6 
n_layer = 6
dropout = 0.2

In [6]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('./input.txt', 'r', encoding='utf-8') as f: 
    text = f.read()

In [7]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from character to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[ch] for ch in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [8]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
# data loading
def get_batch(split):
    # generate a small batch of data inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.Hn = num_heads
        self.Hs = head_size
        self.attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # key, query and value as one batch
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        Hs = self.Hs
        Hn = self.Hn
        B, T, C = x.shape
        q, k, v = self.attn(x.float()).split(n_embd, dim=-1)
        q = q.view(B, T, Hn, Hs).transpose(1, 2) # (B, Hn, T, Hs)
        k = k.view(B, T, Hn, Hs).transpose(1, 2) # (B, Hn, T, Hs)
        v = v.view(B, T, Hn, Hs).transpose(1, 2) # (B, Hn, T, Hs)

        # Calculating attention using flash attention
        if(True):
            out = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0.0, is_causal=True)
        else:
            # compute attention scores ("affinities")
            wei = q @ k.transpose(-2, -1) * Hs**-0.5 # (B, Hn, T, Hs) @ (B, Hn, Hs, T) ---> (B, Hn, T, T)
            wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
            wei = F.softmax(wei, dim=-1) # (B, Hn, T, T)
            wei = self.dropout(wei)
            # perform the weighted aggregation of the values
            out = wei @ v # (B, Hn, T, T) @ (B, Hn, T, Hs) ---> (B, Hn, T, Hs)

        out = out.transpose(1, 2).contiguous().view(B, T, C) # (B, Hn, T, Hs) ---> (B, T, Hn * Hs)
        out = self.proj(out) # from MHA
        out = self.dropout(out) # from MHA
        return out

class FeedForward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd lile
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# super simple bigram model
class GPTModel(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd=n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B, T) tensors of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [12]:
model = GPTModel()
m = model.to(device)

print(f"Parameter count: {sum(p.numel() for p in m.parameters())/1e6} M parameters")

# create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

Parameter count: 10.788929 M parameters


In [13]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3412, val loss 4.3409
step 500: train loss 1.8614, val loss 1.9833
step 1000: train loss 1.5246, val loss 1.7362
step 1500: train loss 1.3831, val loss 1.6242
step 2000: train loss 1.2938, val loss 1.5738
step 2500: train loss 1.2319, val loss 1.5410
step 3000: train loss 1.1772, val loss 1.5347
step 3500: train loss 1.1323, val loss 1.5310
step 4000: train loss 1.0838, val loss 1.5392
step 4500: train loss 1.0337, val loss 1.5597
step 5000: train loss 0.9893, val loss 1.5836


In [14]:
# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))



HASTINGS:
I will not like the field of thee.

KING RICHARD III:
These totard
My Lord of Bushy, that's all, I'll give you:
The leave in the promly debt commission
Made him dishonour hath say'd him for me:
I will on a part way to behold a man,
Will you keep good quench with him.

DUCHESS OF YORK:
Harp, my lord,
While willingn hat my love, have bigged life.
Ye juddy can me! Who's title for all gaze
And bazes him with the deputy? Or they have alived:
Let us enter to do the sullame of high death:
Ou
